In [1]:
# testing objects for n-dim tead

In [2]:
import torch
from botorch.models.gp_regression import ExactGP
from scipy.stats.qmc import LatinHypercube
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [84]:
# test using true function rather than model
dtype = torch.double
def f_test(x):
    return torch.sin(x).sum(dim=1, keepdim=True)
h = 1e-4
x = torch.rand(20, 2, dtype=dtype)
num_data = x.shape[0]
num_dim = x.shape[1]
delta_s = torch.zeros(num_data, num_dim)
for i in range(0, num_data):
    for j in range(0, num_dim):
        dx = torch.zeros(1, num_dim)
        dx[:,j] = h  # set the dimension for the gradient to have a step
        x_lo = x[i] - dx  # element-wise subtract
        x_hi = x[i] + dx  # element-wise addition
        y_lo = f_test(x_lo)  # unsqueeze to match model syntax
        y_hi = f_test(x_hi)  # unsqueeze to match model syntax
        delta_s[i, j] = (y_hi - y_lo)/(2*h)

In [85]:
delta_s

tensor([[0.9844, 0.6934],
        [0.9841, 0.8323],
        [0.6485, 0.7158],
        [0.9577, 0.9746],
        [0.9907, 0.7200],
        [0.9451, 0.9059],
        [0.9012, 0.6410],
        [0.9759, 0.9635],
        [0.9896, 0.9750],
        [0.9964, 0.9930],
        [0.8037, 0.9663],
        [0.9608, 0.7145],
        [0.7672, 0.9873],
        [0.8886, 0.8604],
        [0.9977, 0.7208],
        [0.9830, 0.7669],
        [0.9907, 0.7383],
        [0.9976, 0.9974],
        [0.8876, 0.9699],
        [0.9748, 0.9314]])

In [86]:
torch.cos(x[0])

tensor([0.9844, 0.6934], dtype=torch.float64)

In [75]:
def nfinite_diff(model):
    """get approx gradient at model training points"""
    # assumes 1d input... see implementation in [3] for update to n-D
    h = 1e-4
    x = model.train_inputs[0]
    num_data = x.shape[0]
    num_dim = x.shape[1]
    delta_s = torch.zeros(num_data, num_dim)
    for i in range(0, num_data):
        for j in range(0, num_dim):
            dx = torch.zeros(1, num_dim)
            dx[:,j] = h  # set the dimension for the gradient to have a step
            x_lo = x[i] - dx  # element-wise subtract
            x_hi = x[i] + dx  # element-wise addition
            y_lo = model(x_lo.unsqueeze(0)).mean  # unsqueeze to match model syntax
            y_hi = model(x_hi.unsqueeze(0)).mean  # unsqueeze to match model syntax
            delta_s[i, j] = (y_hi - y_lo)/(2*h)
    return delta_s

In [76]:
dtype = torch.double
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

train_X = torch.rand(20, 2, dtype=dtype)
train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
model_example = SingleTaskGP(train_X, train_Y)
mll_example = ExactMarginalLogLikelihood(model_example.likelihood, model_example)
fitting = fit_gpytorch_mll(mll_example)

In [77]:
temp = nfinite_diff(model_example)

In [60]:
torch.zeros(1,2)

tensor([[0., 0.]])

In [68]:
temp12 = torch.zeros(1, 2)

In [69]:
temp12

tensor([[0., 0.]])

In [70]:
temp12[:,1] = 0.5

In [73]:
temp12

tensor([[0.0000, 0.5000]])

In [74]:
torch.ones(1,2) - temp12

tensor([[1.0000, 0.5000]])

In [83]:
torch.cos(train_X[3])

tensor([0.7887, 0.9993], dtype=torch.float64)

In [79]:
train_X

tensor([[0.9260, 0.4882],
        [0.8499, 0.8118],
        [0.4858, 0.0084],
        [0.6621, 0.0370],
        [0.2010, 0.5979],
        [0.6886, 0.3785],
        [0.5132, 0.5229],
        [0.1427, 0.5575],
        [0.3726, 0.9736],
        [0.5371, 0.0760],
        [0.5911, 0.0370],
        [0.0936, 0.5676],
        [0.7469, 0.9782],
        [0.2958, 0.2547],
        [0.8592, 0.5713],
        [0.0806, 0.6916],
        [0.2510, 0.5761],
        [0.8902, 0.8409],
        [0.6016, 0.9491],
        [0.0627, 0.6578]], dtype=torch.float64)

In [80]:
temp

tensor([[0.5726, 0.8757],
        [0.6051, 0.6663],
        [0.8296, 0.8780],
        [0.8194, 0.9350],
        [0.9806, 0.8212],
        [0.7869, 0.9470],
        [0.8829, 0.8921],
        [0.9549, 0.8237],
        [0.9368, 0.5439],
        [0.8535, 0.9431],
        [0.8394, 0.9284],
        [0.9303, 0.8046],
        [0.6971, 0.4840],
        [0.8754, 0.9253],
        [0.6269, 0.8473],
        [0.9499, 0.7596],
        [0.9765, 0.8418],
        [0.5485, 0.6262],
        [0.8285, 0.5412],
        [0.9318, 0.7694]], grad_fn=<CopySlices>)

In [48]:
torch.sin(torch.tensor(0.8997)) + torch.sin(torch.tensor(0.8082))

tensor(1.5062)

In [49]:
torch.cos(train_X[0])

tensor([0.6219, 0.6908], dtype=torch.float64)

In [50]:
torch.cos(torch.tensor(0.8997))

tensor(0.6218)

In [ ]:
torch.sin(train_X+)